## Text Classification of Rotten Tomato Movie Reviews
Author: James Fung

In [2]:
#Load in the data.
import pandas as pd

reviews = pd.read_csv("/Users/james.fung/Desktop/Test Scripts/Rotten Tomato Reviews/rotten_tomatoes_reviews.csv")

In [24]:
#Let's check out an example of a bad and good review.

#Good review.
print("Good review:" + reviews.iloc[0][1])
print("")
#Bad review.
print("Bad review:" + reviews.iloc[2][1])

Good review: Manakamana doesn't answer any questions, yet makes its point: Nepal, like the rest of our planet, is a picturesque but far from peaceable kingdom.

Bad review: It would be difficult to imagine material more wrong for Spade than Lost & Found.


It seems here that there isn't a clear distinction between a positive and negative review - however I do notice that the bad review contained more "bad" words, such as difficult and wrong.

Let's clean up the reviews a bit by removing stop words and punctuation.

In [39]:
#Remove punctuation.
import string

reviews.iloc[0][1].translate(str.maketrans('','',string.punctuation))

' Manakamana doesnt answer any questions yet makes its point Nepal like the rest of our planet is a picturesque but far from peaceable kingdom'

In [25]:
#Import stop words.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [29]:
stop_words = set(stopwords.words('english'))

In [33]:
word_tokens = word_tokenize(reviews.iloc[0][1])

In [34]:
word_tokens

['Manakamana',
 'does',
 "n't",
 'answer',
 'any',
 'questions',
 ',',
 'yet',
 'makes',
 'its',
 'point',
 ':',
 'Nepal',
 ',',
 'like',
 'the',
 'rest',
 'of',
 'our',
 'planet',
 ',',
 'is',
 'a',
 'picturesque',
 'but',
 'far',
 'from',
 'peaceable',
 'kingdom',
 '.']

In [35]:
filtered_sentence = [w for w in word_tokens if not w in stop_words]

In [36]:
filtered_sentence

['Manakamana',
 "n't",
 'answer',
 'questions',
 ',',
 'yet',
 'makes',
 'point',
 ':',
 'Nepal',
 ',',
 'like',
 'rest',
 'planet',
 ',',
 'picturesque',
 'far',
 'peaceable',
 'kingdom',
 '.']